<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# KNN Classification and Imputation: Cell Phone Churn Data

_Authors: Kiefer Katovich (SF)_

---

In this lab you will practice using KNN for classification (and a little bit for regression as well).

The dataset is one on "churn" in cell phone plans. It has information on the usage of the phones by different account holders and whether or not they churned or not.

Our goal is to predict whether a user will churn or not based on the other features.

We will also be using the KNN model to **impute** missing data. There are a couple of columns in the dataset with missing values, and we can build KNN models to predict what those missing values will most likely be. This is a more advanced imputation method than just filling in the mean or median.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.neighbors import KNeighborsClassifier

### 1. Load the cell phone "churn" data containing some missing values.

In [ ]:
churn = pd.read_csv('../../datasets/churn_missing.csv')

### 2. Examine the data. What columns have missing values?

In [ ]:
churn.head(3)

In [ ]:
churn.shape

In [ ]:
churn.isnull().sum()

In [ ]:
# about 12% ofvmail plan & message are null

In [ ]:
churn.dtypes

In [ ]:
churn.intl_plan.value_counts(dropna=False)

In [ ]:
churn.vmail_plan.value_counts(dropna=False)

In [ ]:
churn.state.value_counts()

In [ ]:
#Note: DC is being counted as a state

### 3. Convert the `vmail_plan` and `intl_plan` colums to binary integer columns.

Make sure that if a value is missing that you don't fill it in with a new value! Preserve the missing values.

In [ ]:
churn.loc[:,'vmail_plan'] = churn.vmail_plan.map(lambda x: 1 if x == 'yes' else 0 if x == 'no' else x)
churn.loc[:,'intl_plan'] = churn.intl_plan.map(lambda x: 1 if x == 'yes' else 0 if x == 'no' else x)

### 4. Create dummy coded columns for state and concatenate it to the churn dataset.

> **Remember:** You will need to leave out one of the state dummy coded columns to serve as the "reference" column since we will be using these for modeling.

In [ ]:
states = pd.get_dummies(churn.state, drop_first=True)
states.head(3)
# drop_first removed Alaska(AK) from being converted to a dummy column

In [ ]:
states.shape

In [ ]:
print(len(churn.state.unique()))

In [ ]:
churn = pd.concat([churn, states], axis=1)

### 5. Create a version of the churn data that has no missing values.

Calculate the shape

In [ ]:
churn_nona = churn.dropna()
churn_nona.shape

### 6. Create a target vector and predictor matrix.

- Target should be the `churn` column.
- Predictor matrix should be all columns except `area_code`, `state`, and `churn`.

In [ ]:
X =churn_nona.drop(['area_code','state','churn'], axis =1)
y = churn_nona.churn.values

### 7. Calculate the baseline accuracy for `churn`.

In [ ]:
churn_nona.churn.mean()
# Less than 0.5

In [ ]:
baseline = 1. - churn_nona.churn.mean()
print(baseline)

### 8. Cross-validate a KNN model predicting `churn`. 

- Number of neighbors should be 5.
- Make sure to standardize the predictor matrix.
- Set cross-validation folds to 10.

Report the mean cross-validated accuracy.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [ ]:
scores = cross_val_score(knn, Xs, y, cv=10)
print(scores)
print(np.mean(scores))

### 9. Iterate from k=1 to k=49 (only odd k) and cross-validate the accuracy of the model for each.

Plot the cross-validated mean accuracy for each score. What is the best accuracy?

In [ ]:
k_values = list(range(1,50,2))
accs = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, Xs, y, cv=10)
    accs.append(np.mean(scores))

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(k_values, accs, lw=3)
plt.show()

print(np.max(accs))

In [ ]:
# looks like there is a peak at about k=7 and then levels out after k=17

### 10. Imputing with KNN

K-Nearest Neighbors can be used to impute missing values in datasets. What we will do is estimate the most likely value for the missing data based on a KNN model.

We have two columns with missing data:
- `vmail_plan`
- `vmail_message`

**10.A: Create two subsets of the churn dataset: one without missing values for `vmail_plan` and `vmail_message`, and one with the missing values.**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
missing_cols = ['vmail_plan','vmail_message']

In [ ]:
impute_missing = churn.loc[churn.vmail_plan.isna(), :]
impute_valid = churn.loc[~churn.vmail_plan.isna(), :]

First we will impute values for `vmail_plan`. This is a categorical column and so we will impute using classification (predicting whether the plan is yes or no, 1 vs. 0).

**10.B: Create a target that is `vmail_plan` and predictor matrix that is all columns except `state`, `area_code`, `churn`, `vmail_plan`, and `vmail_message`.**

> **Note:** We don't include the `churn` variable in the model to impute. Why? We are imputing these missing values so that we can use the rows to predict churn with more data afterwards. If we imputed with churn as a predictor then we would be cheating.

In [ ]:
impute_cols = [c for c in impute_valid.columns if not c in ['state','area_code','churn']+missing_cols]
y = impute_valid.vmail_plan.values
X = impute_valid[impute_cols]

**10.C: Standardize the predictor matrix.**

In [ ]:
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [ ]:
X.columns

**10.D: Find the best K for predicting `vmail_plan`.**

You may want to write a function for this. What is the accuracy for predicting `vmail_plan` at the best K? What is the baseline accuracy for `vmail_plan`?

In [ ]:
def find_best_k_cls(X, y, k_min=1, k_max=51, step=2, cv=5):
    k_range = list(range(k_min, k_max+1, step))
    accs = []
    for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, X, y, cv=cv)
        accs.append(np.mean(scores))
    print(np.max(accs), np.argmax(k_range))
    return np.argmax(k_range)

In [ ]:
find_best_k_cls(Xs, y)

In [ ]:
impute_valid.vmail_plan.mean()
# less than 0.5

In [ ]:
vmail_plan_baseline = 1. - impute_valid.vmail_plan.mean()
print(vmail_plan_baseline)

In [ ]:
# we can already see that out calculated baseline is practically
# identical to the best CV score which leads us down the path
# to believe that there would be very little knowledge gain from
# said model

**10.E: Fit a `KNeighborsClassifier` with the best number of neighbors.**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(Xs, y)

**10.F: Predict the missing `vmail_plan` values using the subset of the data where it is misssing.**

You will need to:
1. Create a new predictor matrix using the same predictors but from the missing subset of data.
- Standardize this predictor matrix *using the StandardScaler object fit on the non-missing data*. This means you will just use the `.transform()` function. It is important to standardize the new predictors the same way we standardized the original predictors if we want the predictions to make sense. Calling `.fit_transform()` will reset the standardized scale.
- Predict what the missing vmail plan values should be.
- Replace the missing values in the original with the predicted values.

> **Note:** It may predict all 0's. This is OK. If you want to see the predicted probabilities of `vmail_plan` for each row you can use the `.predict_proba()` function instead of `.predict()`.  You can use these probabilities to manually set the criteria threshold.

In [ ]:
X_miss = impute_missing[impute_cols]
X_miss_s = ss.transform(X_miss)

In [ ]:
vmail_plan_impute = knn.predict(X_miss_s)

In [ ]:
vmail_plan_impute

In [ ]:
# creating a DF copy to use to imputed missing values
churn_imputed = churn.copy()
# filling missing vmail_plan values with those predicted by KNN model
churn_imputed.loc[churn.vmail_plan.isnull(), 'vmail_plan'] = vmail_plan_impute

### 11. Impute the missing values for `vmail_message` using the same process.

Since `vmail_message` is essentially a continuous measure, you need to use `KNeighborsRegressor` instead of the `KNeighborsClassifier`.

KNN can do both regression and classification! Instead of "voting" on the class like in classification, the neighbors will average their value for the target in regression.


In [ ]:
def find_best_k_reg(X, y, k_min=1, k_max=51, step=2, cv=10):
    k_range = list(range(k_min, k_max+1, step))
    r2s = []
    for k in k_range:
        knn = KNeighborsRegressor(n_neighbors=k)
        scores = cross_val_score(knn, X, y, cv=cv)
        r2s.append(np.mean(scores))
    print(np.max(r2s), np.argmax(k_range))
    return np.argmax(k_range)

In [ ]:
# x-true & y-true
y = impute_valid.vmail_message.values
X = impute_valid[impute_cols]

# set and fit the scaler
ss = StandardScaler()
Xs = ss.fit_transform(X)

# call/use find k-best function on known data
best_k = find_best_k_reg(Xs, y)

# apply k-best to fit model
knn = KNeighborsRegressor(n_neighbors=best_k)
knn.fit(Xs, y)

# prepair rows with missing target values
X_miss = impute_missing[impute_cols]
X_miss_s = ss.transform(X_miss)

# use model to predict unknown values
vmail_message_impute = knn.predict(X_miss_s)
vmail_message_impute

# this r2 is not very good

In [ ]:
# assign predicted values to missing values in dataframe
churn_imputed.loc[churn.vmail_message.isnull(), 'vmail_message'] = vmail_message_impute

### 12. Given the accuracy (and $R^2$) of your best imputation models when finding the best K neighbors, do you think imputing is a good idea?

In [ ]:
# The accuracy and R2 are very bad. This our imputed values are most likely wrong with these models.
# This doesn't necessarily mean that imputation is a bad idea, but we may want to consider
# using a different method.

### 13. With the imputed dataset, cross-validate the accuracy predicting churn. Is it better? Worse? The same?

In [ ]:
X = churn_imputed[[c for c in churn_nona.columns if not c in ['area_code','state','churn']]]
y = churn_imputed.churn.values

ss = StandardScaler()
Xs = ss.fit_transform(X)

k_values = list(range(1,50,2))
accs = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, Xs, y, cv=10)
    accs.append(np.mean(scores))
    
fig, ax = plt.subplots(figsize=(8,5))
ax.plot(k_values, accs, lw=3)
plt.show()

In [ ]:
print(np.max(accs))

In [ ]:
# Its basically exactly the same. 
# However the peak performance comes at about k=9
# Given that our model using imputed data scores almost exactly the same
# and requires more neighbors to come reach optimal accuracy
# we would either want to investigate other means of imputation
# or
# utilize the original model as it is computationally more efficient.